In [1]:
import random
import os
import sys
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import numpy as np
from seriesDataset import seriesDataset
from torchvision import datasets
from torchvision import transforms

In [2]:
# source_root = '/home/iot/jupyter/root_dir/zkx2/letterSim'
source_root = '/home/iot/jupyter/root_dir/zkx2/shapeletX'

labelMap = {"a":0, "b":1, "c":2, "d":3, "e":4}

source_dataset = seriesDataset( source_root, labelMap , False)


dataloader_source = torch.utils.data.DataLoader(
    dataset=source_dataset,
    batch_size=10,
    shuffle=True,
    num_workers=8
)


test_root = '/home/iot/jupyter/root_dir/zkx2/UserData/user1/letter'
test_dataset = seriesDataset(test_root , labelMap , False)
dataloader_test = torch.utils.data.DataLoader(
    dataset= test_dataset,
    batch_size=20,
    shuffle=True,
    num_workers=8
)

In [3]:
import torch.nn as nn

class KaiNet(nn.Module):
    def __init__(self):
        super(KaiNet, self).__init__()
        self.feature = nn.Sequential()
        self.feature.add_module('f_conv1', nn.Conv1d(1,64,kernel_size=3))
        self.feature.add_module('f_bn', nn.BatchNorm1d(64))
        self.feature.add_module('f_relu', nn.ReLU(True))
        self.feature.add_module('f_conv2', nn.Conv1d(64,128,kernel_size=3))
        self.feature.add_module('f_bn2', nn.BatchNorm1d(128))
        self.feature.add_module('f_relu2', nn.ReLU(True))
        self.feature.add_module('f_conv3', nn.Conv1d(128,128,kernel_size=3))
        self.feature.add_module('f_bn3', nn.BatchNorm1d(128))
        self.feature.add_module('f_relu3', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(250*128, 32))
        self.class_classifier.add_module("c_bn1", nn.BatchNorm1d(32))
        self.class_classifier.add_module('c_relu', nn.ReLU(True))
        self.class_classifier.add_module('c_fc2', nn.Linear(32,5))


    def forward(self, input_data ):
        input_data = input_data.view(-1 , 1, 256)
        x = self.feature(input_data.float())
        x = x.view(-1 , 250*128)
        class_output = self.class_classifier(x)

        return class_output



In [4]:
class MyTransformer(nn.Module):
    def __init__(self):
        super(MyTransformer , self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(256, nhead=8, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer , num_layers=1)
        self.classifier = nn.Sequential()
        self.classifier.add_module( 'c_fc1' ,nn.Linear( 256  , 128))
        self.classifier.add_module('c_relu' , nn.ReLU(True))
        self.classifier.add_module( 'c_fc2'  , nn.Linear(128 , 5))
    
    def forward(self, input_data):
        input_data = input_data.view(-1 , 1, 256)
        x = input_data.float()
        x = self.transformer_encoder(x)
        x = x.view(-1 , 256 )
        y = self.classifier(x)
        return y

In [5]:
# my_net = KaiNet()
my_net = MyTransformer()
cuda = True
lr = 1e-4

optimizer = optim.Adam(my_net.parameters(), lr=lr)
loss_class = torch.nn.CrossEntropyLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()

In [6]:
for epoch in range(50):
    data_source_iter = iter(dataloader_source)
    len_dataloader = len(dataloader_source)
    my_net.train()
    for i in range(len_dataloader):
        s_img, s_label = data_source_iter.next()
        
        my_net.zero_grad()
        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
        class_output = my_net(input_data = s_img)
        s_label = s_label.view((s_label.shape[0]))
        err_s_label = loss_class(class_output , s_label)
        err_s_label.backward()
        optimizer.step()
        sys.stdout.write('\r epoch:%d  step:%d  err_s_label:%f'%(epoch, i , err_s_label))
        sys.stdout.flush()

    # test 
    my_net.eval()
    test_iter = iter(dataloader_test)
    len_test =len(dataloader_test)
    n_total = 0
    n_correct = 0
    for j in range(len_test):
        data_test = test_iter.next()
        t_img, t_label = data_test
        batch_size = len(t_label)
        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
        
        class_output = my_net(t_img)
        pred = class_output.data.max(1 , keepdim=True)[1]
        n_correct += pred.eq(t_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size
    accu = n_correct.data.numpy() * 1.0 / n_total
    print("accuracy:%f"%(accu))

 epoch:0  step:1689  err_s_label:0.138408accuracy:0.713333
 epoch:1  step:1689  err_s_label:0.545737accuracy:0.673333
 epoch:2  step:1689  err_s_label:0.279323accuracy:0.653333
 epoch:3  step:1689  err_s_label:0.005854accuracy:0.653333
 epoch:4  step:1689  err_s_label:0.098282accuracy:0.626667
 epoch:5  step:1689  err_s_label:0.551152accuracy:0.646667
 epoch:6  step:1689  err_s_label:0.091403accuracy:0.673333
 epoch:7  step:1689  err_s_label:0.112836accuracy:0.633333
 epoch:8  step:1689  err_s_label:0.011688accuracy:0.706667
 epoch:9  step:1689  err_s_label:0.008248accuracy:0.713333
 epoch:10  step:1689  err_s_label:0.122582accuracy:0.660000
 epoch:11  step:1689  err_s_label:0.024375accuracy:0.633333
 epoch:12  step:383  err_s_label:0.120950

KeyboardInterrupt: 

In [ ]:
test_root = '/home/iot/jupyter/root_dir/zkx2/UserData/user10/letter'
test_dataset = seriesDataset(test_root , labelMap , True)
dataloader_test = torch.utils.data.DataLoader(
    dataset= test_dataset,
    batch_size=20,
    shuffle=True,
    num_workers=8
)
test_iter = iter(dataloader_test)
len_test =len(dataloader_test)
n_total = 0
n_correct = 0
for j in range(len_test):
    data_test = test_iter.next()
    t_img, t_label = data_test
    batch_size = len(t_label)
    if cuda:
        t_img = t_img.cuda()
        t_label = t_label.cuda()
    
    class_output = my_net(t_img)
    pred = class_output.data.max(1 , keepdim=True)[1]
    n_correct += pred.eq(t_label.data.view_as(pred)).cpu().sum()
    n_total += batch_size
accu = n_correct.data.numpy() * 1.0 / n_total
print("accuracy:%f"%(accu))